<a href="https://www.kaggle.com/code/hgvaishnavi/nyc-taxi-fare-prediction?scriptVersionId=234794552" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages
library(geosphere)
library(lubridate)
library(ggplot2)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

As the [dataset](https://www.kaggle.com/c/new-york-city-taxi-fare-prediction/data) contains more than 55 Million rows and it take a lot of time to process, I am using sample data of 100,000 rows.


| **Sampling Method**     | **Representativeness** | **Best For**            | **Notes**                               |
|-------------------------|------------------------|-------------------------|-----------------------------------------|
| Random Sampling         | ⭐⭐⭐⭐⭐                  | Most accurate sampling  | Use shell shuf or clever R chunking     |
| Stratified Sampling     | ⭐⭐⭐⭐⭐                  | Imbalanced datasets     | Needs variable knowledge               |
| Sequential Sampling     | ⭐⭐                     | Quick previews          | Risk of bias                           |
| Systematic Sampling     | ⭐⭐                     | Quick approx. samples   | Risk of missing trends                 |

Used Random Sampling to get sample data because,

✅ Every row in your dataset has an equal chance of being included in the sample.

✅ Most representative of the population if truly random.

❗ Requires access to all row indices, which can be memory-heavy in R.

Use the shuf command to randomly select rows from the CSV.
shuf is a command-line utility available on Unix/Linux/macOS (and can be enabled in Windows via WSL or Git Bash).

📌 Command format on git bash:

shuf -n [sample_size] your_file.csv > sample.csv

This is how sample.csv was created.

In [ ]:
library(data.table)

unclean_data <- fread("/kaggle/input/nyc-taxi-fare-sample-dataset/sample.csv")
clean_data <- unclean_data

In [ ]:
summary(unclean_data)

If you observe the above summary,

*fare_amount:*
<br>Based on [this discussion](https://www.kaggle.com/c/new-york-city-taxi-fare-prediction/discussion/63319), the taxi fare within NYC is between 2.5 and 100.
<br>Hence any record that is not in this range, must be cleared.

*pickup_datetime:*
<br>Datatype must be changed.

*passenger_count:*
<br>No.of passengers cannot be less than 1.
<br>Max. capacity of taxi is 6.
<br>Hence any record that is not in this range, must be cleared.

*pickup and drop-off locations:*
<br>NYC Coordinates:
| **Type**     | **Approximate Range**  |
|--------------|------------------------|
| Latitude     | 40.5 to 41.0           |
| Longitude    | -74.25 to -73.70       |
<br>Any point that is outside of NYC, must be removed.
<br>Rides having same pickup and dropoff location must be removed.


#### Remove Null Values

In [ ]:
sum(!complete.cases(unclean_data))

In [ ]:
clean_data <- na.omit(unclean_data)

#### Remove records having same pickup and dropoff locations

In [ ]:
nrow(clean_data[
  pickup_latitude == dropoff_latitude &
    pickup_longitude == dropoff_longitude])

In [ ]:
clean_data <- clean_data[!(pickup_latitude == dropoff_latitude & 
                              pickup_longitude == dropoff_longitude)]

#### Remove records with irrelevent fare amount

In [ ]:
nrow(clean_data[fare_amount < 2.5 | fare_amount > 100])

In [ ]:
clean_data <- clean_data[fare_amount >= 2.5 & fare_amount <= 100]

#### Remove records with irrelevent passenger count

In [ ]:
nrow(clean_data[passenger_count > 6 | passenger_count < 1])

In [ ]:
clean_data <- clean_data[passenger_count > 0 & passenger_count <= 6]

#### Remove irrelevent Pickup Locations

In [ ]:
nrow(clean_data[
  pickup_latitude < 40.5 | pickup_latitude > 41 |
    pickup_longitude < -74.25 | pickup_longitude > -73.7])

In [ ]:
clean_data <- clean_data[
  pickup_latitude >= 40.5 & pickup_latitude <= 41 & 
    pickup_longitude >= -74.25 & pickup_longitude <= -73.7
]

#### Remove irrelevent Dropoff Locations

In [ ]:
nrow(clean_data[
  dropoff_latitude < 40.5 | dropoff_latitude > 41 |
    dropoff_longitude < -74.25 | dropoff_longitude > -73.7])

In [ ]:
clean_data <- clean_data[
    dropoff_latitude >= 40.5 & dropoff_latitude <= 41 & 
        dropoff_longitude >= -74.25 & dropoff_longitude <= -73.7
]

#### Remove irrelevent distances

🚕 In real life:
<br>A taxi can be taken for very short distances, like 1–2 blocks.
<br>This might be as low as 0.1 to 0.3 km (100–300 meters).

<br>A typical maximum distance for NYC taxis would be:
<br>Around 30–50 km within NYC
<br>JFK to Manhattan is ~30 km
<br>Some trips may go even outside NYC (e.g., to Newark)
<br>So, realistic max trip distances could be up to 60 km, but anything beyond that is likely an outlier or error.

So, lets create a new column trip distance in km and remove the records where distance is less than 100m and greater than 60km 

In [ ]:
clean_data$trip_distance_km <- distHaversine(
  matrix(c(clean_data$pickup_longitude, clean_data$pickup_latitude), ncol = 2),
  matrix(c(clean_data$dropoff_longitude, clean_data$dropoff_latitude), ncol = 2)
) / 1000

In [ ]:
summary(clean_data$trip_distance_km)

In [ ]:
sum(clean_data$trip_distance_km < 0.1 | clean_data$trip_distance_km > 60, na.rm = TRUE)

In [ ]:
clean_data <- clean_data[clean_data$trip_distance_km >= 0.1 & clean_data$trip_distance_km <= 60,  ]

#### Change the datatype of pickup_datetime

In [ ]:
clean_data$pickup_datetime <- as.POSIXct(clean_data$pickup_datetime, format="%Y-%m-%d %H:%M:%S")

In [ ]:
summary(clean_data)

In [ ]:
nrow(clean_data)

### Visualizations

In [ ]:


# fare amount
hist(unclean_data$fare_amount, main = "Histogram of Fare Amount (Unclean Data)", xlab = ("Fare Amount"))
hist(clean_data$fare_amount, main = "Histogram of Fare Amount (Clean Data)", xlab = ("Fare Amount"))

# passenger count
ggplot(unclean_data, aes(x = factor(passenger_count))) +
  geom_bar(fill = "steelblue") +
  labs(
    title = "Distribution of Passenger Counts (Unclean Data)",
    x = "Passenger Count",
    y = "Number of Rides"
  ) +
  theme_minimal()

ggplot(clean_data, aes(x = factor(passenger_count))) +
  geom_bar(fill = "steelblue") +
  labs(
    title = "Distribution of Passenger Counts (Unclean Data)",
    x = "Passenger Count",
    y = "Number of Rides"
  ) +
  theme_minimal()


# NYC plot for pickup

ggplot(unclean_data, aes(x = pickup_longitude, y = pickup_latitude)) +
  geom_point(alpha = 0.1, color = "darkgreen", size = 0.2) +
  coord_fixed(1.2) +
  labs(title = "Pickup Locations (Unclean Data)", x = "Longitude", y = "Latitude") +
  theme_minimal()

# NYC plot for dropoff

ggplot(unclean_data, aes(x = pickup_longitude, y = pickup_latitude)) +
  geom_point(alpha = 0.1, color = "darkred", size = 0.2) +
  coord_fixed(1.2) +
  labs(title = "Dropoff Locations (Unclean Data)", x = "Longitude", y = "Latitude") +
  theme_minimal()



# NYC plot for pickup

ggplot(clean_data, aes(x = pickup_longitude, y = pickup_latitude)) +
  geom_point(alpha = 0.1, color = "darkgreen", size = 0.5) +
  coord_fixed(1.2) +
  labs(title = "Pickup Locations (Clean Data)", x = "Longitude", y = "Latitude") +
  theme_minimal()

# NYC plot for dropoff

ggplot(clean_data, aes(x = pickup_longitude, y = pickup_latitude)) +
  geom_point(alpha = 0.1, color = "darkred", size = 0.5) +
  coord_fixed(1.2) +
  labs(title = "Dropoff Locations (Clean Data)", x = "Longitude", y = "Latitude") +
  theme_minimal()

In [ ]:
library(leaflet)

set.seed(42)  # for reproducibility

      # pickup
leaflet(clean_data) %>%
  addTiles() %>%
  addCircleMarkers(
    lng = ~pickup_longitude,
    lat = ~pickup_latitude,
    radius = 1,
    color = "green",
    opacity = 0.5,
    popup = ~paste("Fare: $", fare_amount)
  ) %>%
  addLegend("bottomright", colors = "green", labels = "Pickup Locations")

      # dropoff
leaflet(clean_data) %>%
  addTiles() %>%
  addCircleMarkers(
    lng = ~dropoff_longitude,
    lat = ~dropoff_latitude,
    radius = 0.5,
    color = "red",
    opacity = 0.5,
    popup = ~paste("Fare: $", fare_amount)
  ) %>%
  addLegend("bottomright", colors = "red", labels = "Dropoff Locations")

In [ ]:
# Passenger Count Distribution

ggplot(clean_data, aes(x = as.factor(passenger_count))) +
  geom_bar(fill = "steelblue") +
  labs(title = "Passenger Count Distribution",
       x = "Passenger Count",
       y = "Number of Rides") +
  theme_minimal()



# Fare Amount Distribution

ggplot(clean_data, aes(x = fare_amount)) +
  geom_histogram(binwidth = 2, fill = "darkgreen", color = "white") +
  labs(title = "Fare Amount Distribution",
       x = "Fare ($)",
       y = "Number of Rides") +
  theme_minimal()



# Trip Distance

clean_data$distance_bin <- cut(
  clean_data$trip_distance_km,
  breaks = c(0, 5, 10, 15, 20, 25, 30, 35, 40, 45, Inf),
  labels = c("0-5", "5-10", "10-15", "15-20", "20-25", 
             "25-30", "30-35", "35-40", "40-45", "45+"),
  right = FALSE
)

ggplot(clean_data, aes(x = distance_bin)) +
  geom_bar(fill = "blue") +
  labs(title = "Trip Distance Distribution",
       x = "Trip Distance (km)",
       y = "Number of Trips") +
  theme_minimal()



# Fare vs Distance Relationship

ggplot(clean_data, aes(x = trip_distance_km, y = fare_amount)) +
  geom_point(alpha = 0.3, color = "darkgreen") +
  labs(title = "Fare vs Trip Distance",
       x = "Trip Distance (km)",
       y = "Fare Amount ($)") +
  theme_minimal()